In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import matplotlib as mpl
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
from datetime import datetime, timedelta
import warnings
import datetime
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)
mpl.style.use('seaborn')
pd.set_option('display.max_columns', 1000)
from sklearn.metrics import accuracy_score


In [2]:
df=pd.read_csv("aud_usd_h4_h1_all_merged_corr_0.01.csv")
df.columns = df.columns.str.replace('.','_')

In [3]:
df.head()

Unnamed: 0           time_local  target1_x  target2_x  target3_x  \
0           0  2010-01-05 18:00:00     0.9117     0.9135     0.9142   
1           1  2010-01-05 22:00:00     0.9135     0.9142     0.9142   
2           2  2010-01-06 02:00:00     0.9142     0.9142     0.9156   
3           3  2010-01-06 06:00:00     0.9142     0.9156     0.9184   
4           4  2010-01-06 10:00:00     0.9156     0.9184     0.9196   

   bid_close_x  Volatility_USD_hourly  PC_macro_economic_ind_USD_hourly  \
0       0.9124                    2.0                         -2.477564   
1       0.9117                    2.0                         -2.477564   
2       0.9135                    3.0                          0.000000   
3       0.9142                    3.0                          0.000000   
4       0.9142                    0.0                          0.000000   

   Volatility_USD_daily  PC_macro_economic_ind_USD_daily  \
0                   2.0                        -2.477564   
1                   2.0                        -2.477564   
2                   1.5                        -0.395704   
3                   1.5                        -0.395704   
4                   1.5                        -0.395704   

   Volatility_AUD_hourly  PC_macro_economic_ind_AUD_hourly  \
0                    0.0                          0.000000   
1                    0.0                          0.000000   
2                    0.5                         -1.543434   
3                    0.5                         -1.543434   
4                    0.5                         -1.543434   

   PC_macro_economic_ind_AUD_daily  ANZ Job Advertisements_PC_AUD_hourly  \
0                        -1.050000                                   0.0   
1                        -1.050000                                   0.0   
2                        -1.543434                                   0.0   
3                        -1.543434                                   0.0   
4                        -1.543434                                   0.0   

   Building Permits (MoM)_PC_AUD_hourly  Building Permits (YoY)_PC_AUD_hourly  \
0                              0.000000                              0.000000   
1                              0.000000                              0.000000   
2                             -4.484536                              1.397668   
3                             -4.484536                              1.397668   
4                             -4.484536                              1.397668   

   CFTC AUD NC net positions_PC_AUD_hourly  \
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                                      0.0   

   Consumer Price Index (QoQ)_PC_AUD_hourly  \
0                                       0.0   
1                                       0.0   
2                                       0.0   
3                                       0.0   
4                                       0.0   

   Dwelling Unit Starts (QoQ)_PC_AUD_hourly  \
0                                       0.0   
1                                       0.0   
2                                       0.0   
3                                       0.0   
4                                       0.0   

   Employment Change s_a__PC_AUD_hourly  \
0                                   0.0   
1                                   0.0   
2                                   0.0   
3                                   0.0   
4                                   0.0   

   Export Price Index (QoQ)_PC_AUD_hourly  Exports_PC_AUD_hourly  \
0                                     0.0                    0.0   
1                                     0.0                    0.0   
2                                     0.0                    0.0   
3                                     0.0                    0.0   
4    

In [4]:
df=df.dropna(how='all',axis=1)
df.shape

(14028, 1064)

In [5]:
df['year']=pd.DatetimeIndex(df['time_local']).year
df['month']=pd.DatetimeIndex(df['time_local']).month
df['day']=pd.DatetimeIndex(df['time_local']).day
df['hour']=pd.DatetimeIndex(df['time_local']).hour
df['dayofweek']=pd.DatetimeIndex(df['time_local']).dayofweek



In [6]:
import datetime   
#date = datetime.date.today()
date=pd.datetime.now().strftime("%Y-%m-%d %I:%M:%S")
date

'2019-04-17 09:42:54'

In [7]:
df['today']=date
df['today'] =pd.DatetimeIndex(df['today'])

In [8]:
df['nb_months'] = ((df['today'] - pd.to_datetime(df['time_local'] ))/np.timedelta64(1, 'M'))
df['nb_months'] = df['nb_months'].astype(int)

In [9]:
df['class_target_7'] = ((df['target1_x']-df['close_h1_7'])/df['close_h1_7'])*100
df.loc[df['class_target_7'] < 0.0005, 'new_class_target'] = -1
df.loc[df['class_target_7'] > 0.0005, 'new_class_target'] = 1
df.loc[df['class_target_7'] == 0, 'new_class_target'] = 0
#class_target3

In [10]:
col_drop= df.filter(regex='time').columns

In [11]:
target_cols = df.filter(regex='target').columns

In [12]:
cols_x = df.filter(regex='_x').columns

In [13]:
cols_y = df.filter(regex='_y').columns

In [14]:
X=df.drop(['Unnamed: 0','time_local', 'target2_x', 'target3_x', 'target1_y', 'target2_y',
       'target3_y', 'class_target2', 'class_target3',
       'PC_AUD_H4_target1', 'bid_close_y', 'today','target1_x'],axis=1)

In [15]:
X=X.fillna(0)

In [16]:
df_temp=X.select_dtypes(include=['object'])

In [17]:
obj=df_temp.loc[:, df_temp.dtypes =='object']
for col in obj:
    obj[col] = pd.to_numeric(obj[col], errors='coerce')
df_temp[obj.columns]=obj

In [18]:
df_temp=df_temp.replace([np.inf, -np.inf], np.nan)
df_temp=df_temp.fillna(0)
X[df_temp.columns]=df_temp

In [19]:
X_final= X[X['year'] < 2018]
s=X[['year','month', 'dayofweek']]
for each in s:
    dummies = pd.get_dummies(s[each], prefix=each, drop_first=False)
    X_final=X_final.drop([each],axis=1)
    X_final = pd.concat([X_final, dummies], axis=1)

In [20]:
X_final=X_final[X_final.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]

In [22]:
y=X_final['new_class_target']
X1=X_final.drop(['class_target1','new_class_target','class_target_7'],axis=1)

In [23]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X1)
X1=scaler.transform(X1)

In [24]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
np.seed=0
from sklearn.linear_model import LogisticRegression

In [25]:
from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=1)
X2, y1 = smt.fit_sample(X1, y)
len(X2),len(y1)

(17208, 17208)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X2, y1, test_size=0.3,random_state=1,stratify=y1)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)


(12045, 1082) (12045,)
(5163, 1082) (5163,)


In [27]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier()
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

        -1.0       0.49      0.51      0.50      1721
         0.0       0.95      0.91      0.93      1721
         1.0       0.48      0.48      0.48      1721

   micro avg       0.63      0.63      0.63      5163
   macro avg       0.64      0.63      0.64      5163
weighted avg       0.64      0.63      0.64      5163



In [30]:
print(classification_report(y_train,gb.predict(X_train)))

              precision    recall  f1-score   support

        -1.0       0.70      0.74      0.72      4015
         0.0       0.96      0.90      0.93      4015
         1.0       0.71      0.71      0.71      4015

   micro avg       0.78      0.78      0.78     12045
   macro avg       0.79      0.78      0.79     12045
weighted avg       0.79      0.78      0.79     12045



In [31]:
accuracy_score(y_test,y_pred)

0.6339337594421848

In [32]:
accuracy_score(y_train,gb.predict(X_train))

0.7848899958489

In [44]:
pd.crosstab(y_test,y_pred)

col_0  -1.0   0.0   1.0
row_0                  
-1.0    580   536   561
 0.0    614   562   580
 1.0    619   540   571

In [45]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=80)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)

In [46]:
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5440635289560333, 0.5712743877127439)

In [48]:

from sklearn.model_selection import GridSearchCV
parameters = {
    "learning_rate": [0.1],
    "max_depth":[3,5,8],
    "criterion": ["friedman_mse"],
    "subsample":[0.8,0.9,1.0],
    "n_estimators":[90,100,120]
    }

clf = GridSearchCV(gb, parameters, cv=5,verbose=5)
clf.fit(X_train,y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8, score=0.6331950207468879, total= 5.9min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.9min remaining:    0.0s


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8, score=0.6355334163553341, total= 6.6min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8 


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 18.7min remaining:    0.0s


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8, score=0.6081361560813615, total= 6.9min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 25.7min remaining:    0.0s


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8, score=0.632890365448505, total= 5.5min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9, score=0.6286307053941909, total= 6.3min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9, score=0.6255707762557078, total= 7.0min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9, score=0.6334578663345787, total= 5.8min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9,

KeyboardInterrupt: 

In [49]:

from sklearn.model_selection import GridSearchCV
parameters = {
    "learning_rate": [0.1],
    "max_depth":[3,5,8],
    "criterion": ["friedman_mse"],
    "subsample":[0.8,0.9],
    "n_estimators":[100,120]
    }

clf = GridSearchCV(gb, parameters, cv=5,verbose=3)
clf.fit(X_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8, score=0.6261410788381743, total= 7.8min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.8min remaining:    0.0s


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8, score=0.6247405562474055, total= 7.6min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 15.3min remaining:    0.0s


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8, score=0.6330427563304276, total= 7.9min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8, score=0.6164383561643836, total= 8.1min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8, score=0.6399501661129569, total= 7.9min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.9, score=0.637759336099585, total= 8.5min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsa

KeyboardInterrupt: 

In [52]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.1,subsample=0.8,n_estimators=120)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.6380011621150494, 0.8048152760481527)

In [53]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.1,subsample=0.8,n_estimators=90)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.6261863257795856, 0.7813200498132005)

In [56]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.1,subsample=0.6,n_estimators=100)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.6246368390470657, 0.7901203819012038)

In [58]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.6,n_estimators=100)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5541351927174124, 0.593607305936073)

In [59]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.6,n_estimators=100,max_depth=4)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5828006972690296, 0.6596928185969282)

# Final model

In [72]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.6,n_estimators=120,max_depth=3)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5690490025179159, 0.6099626400996264)

In [73]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

        -1.0       0.43      0.61      0.50      1677
         0.0       0.94      0.69      0.80      1756
         1.0       0.47      0.41      0.44      1730

   micro avg       0.57      0.57      0.57      5163
   macro avg       0.61      0.57      0.58      5163
weighted avg       0.62      0.57      0.58      5163



In [74]:
print(classification_report(y_train,gb.predict(X_train)))

              precision    recall  f1-score   support

        -1.0       0.48      0.66      0.56      4059
         0.0       0.94      0.70      0.80      3980
         1.0       0.53      0.47      0.50      4006

   micro avg       0.61      0.61      0.61     12045
   macro avg       0.65      0.61      0.62     12045
weighted avg       0.65      0.61      0.62     12045



In [75]:
pd.crosstab(y_test,y_pred)

col_0  -1.0   0.0   1.0
row_0                  
-1.0   1016    36   625
 0.0    358  1217   181
 1.0    979    46   705

In [76]:
pd.crosstab(y_train,gb.predict(X_train))

col_0  -1.0   0.0   1.0
row_0                  
-1.0   2669    78  1312
 0.0    842  2795   343
 1.0   2015   108  1883

In [61]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.5,n_estimators=110,max_depth=4)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5818322680612047, 0.6658364466583645)

In [62]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.6,n_estimators=100,max_depth=4,max_features="auto")
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5719542901413907, 0.6529680365296804)

In [63]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.6,n_estimators=120,max_depth=4,max_features="auto")
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5888049583575441, 0.6751349107513491)

In [65]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.4,n_estimators=150,max_depth=4,max_features="auto")
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5986829362773581, 0.7070983810709838)

In [66]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

(0.6226999806314159, 0.7632212536322125)

In [67]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

        -1.0       0.47      0.53      0.49      1677
         0.0       0.95      0.89      0.92      1756
         1.0       0.48      0.45      0.46      1730

   micro avg       0.62      0.62      0.62      5163
   macro avg       0.63      0.62      0.62      5163
weighted avg       0.63      0.62      0.63      5163



In [68]:
print(classification_report(y_train,gb.predict(X_train)))

              precision    recall  f1-score   support

        -1.0       0.59      0.71      0.65      4059
         0.0       0.93      0.83      0.88      3980
         1.0       0.64      0.58      0.61      4006

   micro avg       0.71      0.71      0.71     12045
   macro avg       0.72      0.71      0.71     12045
weighted avg       0.72      0.71      0.71     12045



In [69]:
from xgboost import XGBClassifier
model = XGBClassifier(eta=0.01,subsample=0.8,colsample_bytree=0.8)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

(0.6225062947898509, 0.765296803652968)

In [71]:
from xgboost import XGBClassifier
model = XGBClassifier(eta=0.01,subsample=0.8,colsample_bytree=0.8,lambda=0.8)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

SyntaxError: invalid syntax (<ipython-input-71-000e11badc3a>, line 2)